202224111120
段志清

In [1]:
import nltk
nltk.download('movie_reviews')
nltk.download('punkt')

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\Museum\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\movie_reviews.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Museum\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [6]:
from nltk.corpus import movie_reviews
from nltk.classify import NaiveBayesClassifier
from nltk.classify.util import accuracy as nltk_accuracy

def extract_features(words):
    return dict([(word, True) for word in words])

def analyze_threshold_impact(classifier, reviews, pos_thresholds, neg_thresholds):
    for pos_thresh in pos_thresholds:
        for neg_thresh in neg_thresholds:
            results = {'Positive': 0, 'Negative': 0, 'Neutral': 0}
            for review in reviews:
                words = review.split()
                features = extract_features(words)
                probabilities = classifier.prob_classify(features)
                prob_pos = probabilities.prob('Positive')
                prob_neg = probabilities.prob('Negative')
                
                if prob_pos >= pos_thresh:
                    results['Positive'] += 1
                elif prob_neg >= neg_thresh:
                    results['Negative'] += 1
                else:
                    results['Neutral'] += 1

if __name__ == '__main__':
    # 加载和准备数据
    fileids_pos = movie_reviews.fileids('pos')
    fileids_neg = movie_reviews.fileids('neg')

    features_pos = [(extract_features(movie_reviews.words(fileids=[f])), 'Positive') for f in fileids_pos]
    features_neg = [(extract_features(movie_reviews.words(fileids=[f])), 'Negative') for f in fileids_neg]

    # 分割数据集
    threshold = 0.8
    num_pos = int(threshold * len(features_pos))
    num_neg = int(threshold * len(features_neg))

    features_train = features_pos[:num_pos] + features_neg[:num_neg]
    features_test = features_pos[num_pos:] + features_neg[num_neg:]

    # 训练分类器
    classifier = NaiveBayesClassifier.train(features_train)

    # 输入审查以供演示
    input_reviews = [
        'This is a good movie, it has beautiful pictures',
        'The expression of this movie is so bad that I do not like it',
        'The framework and content of the film are very substantial',
        'The performance of the actors in the film is so bad'
    ]

    # 演示使用不同阈值进行分类
    test_thresholds = [0.5, 0.6, 0.7, 0.8]
    analyze_threshold_impact(classifier, input_reviews, test_thresholds, test_thresholds)

    # 显示带有样本阈值的详细预测
    for review in input_reviews:
        print("\nMovie Review:", review)
        features = extract_features(review.split())
        probabilities = classifier.prob_classify(features)
        prob_pos = round(probabilities.prob('Positive'), 2)
        prob_neg = round(probabilities.prob('Negative'), 2)
        
        # 应用阈值
        if prob_pos >= 0.6:
            sentiment = 'Positive'
        elif prob_neg >= 0.6:
            sentiment = 'Negative'
        else:
            sentiment = 'Neutral'
        
        print(f"Predicted result: {sentiment}")
        print(f"Probability: {prob_pos if prob_pos > prob_neg else prob_neg}")


Movie Review: This is a good movie, it has beautiful pictures
Predicted result: Positive
Probability: 0.68

Movie Review: The expression of this movie is so bad that I do not like it
Predicted result: Negative
Probability: 0.7

Movie Review: The framework and content of the film are very substantial
Predicted result: Positive
Probability: 0.67

Movie Review: The performance of the actors in the film is so bad
Predicted result: Neutral
Probability: 0.57
